In [9]:
import silica as si
import magma as m
from silica import bits, add
from magma.bit_vector import BitVector
from magma.testing.coroutine import check


@si.coroutine
def Counter(width, init=0, incr=1):
    value = bits(init, width)
    O = value
    cout = False
    while True:
        yield O, cout
        O = value
        value, cout = add(value, bits(incr, width), cout=True)
        
si.compile(Counter(4), file_name="build/silica_counter.py")
with open("build/silica_counter.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator

Counter = DefineCircuit("Counter", "O", Out(Bits(4)), "cout", Out(Bit), *ClockInterface(has_ce=False))
CE = VCC
value = Register(4, has_ce=True)
value_CE = [CE]
wireclock(Counter, value)
value_next_0_tmp = [value.O[__silica_i] for __silica_i in range(4)]
O_0_tmp = value.O
value_next_0_tmp, cout_0_tmp = add(value.O, bits(1, 4), cout=True)
for __silica_i in range(4):
    wire(value_next_0_tmp[__silica_i], value.I[__silica_i])
wire(O_0_tmp, Counter.O)
wire(cout_0_tmp, Counter.cout)
if len(value_CE) == 1:
    wire(value_CE[0], value.CE)
else:
    wire(and_(*value_CE), value.CE)
EndDefine()


In [11]:
!magma -o verilog -m lattice -t Counter build/silica_counter.py
!yosys -p 'synth_ice40 -top Counter -blif build/silica_counter.blif' build/silica_counter.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -q -d 1k -o build/silica_counter.txt build/silica_counter.blif
!icetime -tmd hx1k build/silica_counter.txt | grep -B 2 "Total path delay"

import mantle lattice ice40
import mantle lattice mantle40
compiling Register4CE
compiling FullAdder
compiling Add4Cout
compiling Counter
2.27. Printing statistics.

=== Counter ===

   Number of wires:                 47
   Number of wire bits:             68
   Number of public wires:          47
   Number of public wire bits:      68
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 11
     SB_CARRY                        3
     SB_DFFE                         4
     SB_LUT4                         4

Total number of logic levels: 5
Total path delay: 3.23 ns (309.79 MHz)


In [12]:
import os
os.environ["MANTLE"] = "lattice"
from magma import compile
from mantle import DefineCounter
compile("build/mantle_counter", DefineCounter(4))

compiling FullAdder
compiling Add4Cout
compiling Register4
compiling Counter4


In [13]:
!yosys -p 'synth_ice40 -top Counter4 -blif build/mantle_counter.blif' build/mantle_counter.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -q -d 1k -o build/mantle_counter.txt build/mantle_counter.blif
!icetime -tmd hx1k build/mantle_counter.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== Counter4 ===

   Number of wires:                 47
   Number of wire bits:             68
   Number of public wires:          47
   Number of public wire bits:      68
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                 11
     SB_CARRY                        3
     SB_DFF                          4
     SB_LUT4                         4

Total number of logic levels: 5
Total path delay: 3.23 ns (309.79 MHz)
